In [1]:
import os

# --- 1. เคลียร์โฟลเดอร์เก่า (กรณีรันซ้ำ) ---
if os.path.exists("/kaggle/working/3DSCAN"):
    print("🧹 กำลังลบโฟลเดอร์เก่าเพื่ออัปเดต...")
    !rm -rf "/kaggle/working/3DSCAN"

# --- 2. ดึงโค้ดล่าสุดจาก GitHub ---
print("📥 กำลัง Clone Repo: PRIDA-TAKON/3DSCAN ...")
!git clone https://github.com/PRIDA-TAKON/3DSCAN.git

# --- 3. ย้ายพื้นที่ทำงานเข้าไปในโฟลเดอร์ ---
os.chdir("/kaggle/working/3DSCAN")
print(f"✅ ย้าย Working Directory มาที่: {os.getcwd()}")

# --- 4. เช็คไฟล์ข้างใน ---
print("📂 รายชื่อไฟล์ในโปรเจค:")
!ls -lh


📥 กำลัง Clone Repo: PRIDA-TAKON/3DSCAN ...
Cloning into '3DSCAN'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 26 (delta 8), reused 12 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (26/26), 65.00 KiB | 2.41 MiB/s, done.
Resolving deltas: 100% (8/8), done.
✅ ย้าย Working Directory มาที่: /kaggle/working/3DSCAN
📂 รายชื่อไฟล์ในโปรเจค:
total 36K
-rw-r--r-- 1 root root  18K Jan 22 06:11 3d-scan.ipynb
-rw-r--r-- 1 root root  11K Jan 22 06:11 3d-scan.py
-rw-r--r-- 1 root root 1.1K Jan 22 06:11 LICENSE


In [2]:
# เปลี่ยน 'ชื่อไฟล์.py' เป็นชื่อไฟล์จริงที่คุณเห็นจากผลลัพธ์ข้างบน
!python 3d-scan.py

✅ Imports complete
🔍 Checking GPU availability...
✅ GPU Detected: Tesla T4
⏳ Installing dependencies...
🚀 Running: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.8.4 requires fastcore<1.9,>=1.8.0, but you have fastcore 1.11.3 which is incompatible.
🚀 Running: pip install torch torchvision
🚀 Running: pip install nerfstudio
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pypro